In [0]:
import pandas as pd
import numpy as np
import re

In [0]:
#extract data#
#read data, get first dict, prep for cleaning
with open("RESULT","r") as f:
    l=f.readlines()[0]
    first_dict=l.split("}}")

In [0]:
def clean_num(g):
    #use regex, get length：（a x b x c mm OR amm x bmm x c mm）
    pattern = re.compile("((([0-9]+)|([0-9]+\.[0-9]+))(x|[a-z]+x)(([0-9]+)|([0-9]+\.[0-9]+))(x|[a-z]+x)(([0-9]+)|([0-9]+\.[0-9]+))([a-z]+))")
    pattern2=re.compile("((([0-9]+)|([0-9]+\.[0-9]+))(x|[a-z]+x)(([0-9]+)|([0-9]+\.[0-9]+))([a-z]+))")#items with 2 lengths
    g=g.replace(" ","")
    print("g",g)
    e = list(pattern.findall(g))
    e2=list(pattern2.findall(g))
    if e!=[]:
        data=list(e[0])[0]
        data=data.replace("or","")
    elif e2!=[]:
        data=list(e2[0])[0]
        data=data.replace("or","")
    else:#if regex cannot extract it, it might be weight, get weight by ...
        g0=g.split("(")[-1]
        g0=g0.split(")")[0]
        g0=g0.split("or")[-1]
        data=g0
        #print("g0",data)

    #after values extracted, change units
    data=data.replace("kg","*1000 g")#kg-->g
    data=data.replace("grams","g")
    data = data.replace("inches", "in")
    data = data.replace("pounds", "*16 ounces")#pounds-->ounces
    data = data.replace("cm", "*1000 mm")#cm-->mm
    data=data.replace("in","*25.4 mm")#inches-->mm
    data=data.replace("ounces","*28.35 g")#ounce-->g
    if data.endswith("g"):#if it's weight, use * to separate, then compute
        type="g"
        datac=data.replace("g","")
        if "*" in datac:
            data=str(float(datac.split("*")[0])*float(datac.split("*")[1]))
        else:
            data=datac
    if data.endswith("mm"):#compute length..
        type="mm"
        datac=data.replace("mm","")
        data=[]
        if "*" in datac:#if the original unit is not mm but transformed to mm
            datax=datac.split("x")#use x to separate
            for dx in datax:
                if "*" in dx:#if there is *, for example amm x bmm x c mm
                    data.append(str(float(dx.split("*")[0])*float(dx.split("*")[1])))
                else:#if there is no * for example a x b x c mm
                    data.append(str(float(dx)*25.4))
        else:#original unit is mm
            data=datac.split("x")
    print(data)
    print(type)
    return data,type

In [0]:
names=[]
weights=[]
sizes=[]
colunms=[]
for i in first_dict[:-1]:#get all columns which are keys of dict
    #3 parts of data: frist is names，second is weight，thrid is size
    # there is no secondary dict under names; there may be secondary dict under weight OR size
    name=(i.split("': {'weight': ")[0]).split("'")[-1]
    weight=eval((i.split("': {'weight': ")[-1]).split(", 'size': ")[0])
    size=eval((i.split("': {'weight': ")[-1]).split(", 'size': ")[-1]+"}")
    names.append(name)
    weights.append(weight)
    sizes.append(size)#split, get all the names、weight and size
    colunms.extend(list(weight.keys()))#get keys of all the secondary dict of weight
    colunms.extend(list(size.keys()))#get all the keys under secondary dict of size
colunms=list(set(colunms))#rip off duplicates columns
print(colunms,len(weights),len(sizes))
all_data=[['name']]
all_data[0].extend(colunms)#alldata is the first row of excel, rowname

['Shipping Weight', 'Item Dimensions  L x W x H', 'Item Weight', 'Package Dimensions', 'Product Dimensions', 'Weight', 'Size', 'Dimensions'] 86 86


In [0]:
for i in range(len(weights)):
    row=[""]*len(colunms)#a table of length = len（columns），values = “”
    #print("row",row)
    for wk in weights[i].keys():#wk is the key to all secondary dict under weight
        #print("wk",wk)
        for j in range(len(colunms)):
            #print("j",j)
            if wk==colunms[j]:#if values of secondary dict of weight = len(column), then replace value in the column to value in secondary dict of weight
                row[j],type=clean_num(weights[i][wk])#use clean——num, clean value in dict
                #print("row",row)
    for sk in sizes[i].keys():
        #print("wk",wk)
        for j in range(len(colunms)):
            #print("j",j)
            if sk==colunms[j]:
                row[j],type=clean_num(sizes[i][sk])#same as above
    row.insert(0,names[i])
    all_data.append(row)#alldata，add to excel as row
data=pd.DataFrame(all_data)# transform to dataframe
data.to_excel("data00.xlsx",header=False, index=False)#save to excel
#--------------------The End--------------------------#

g 53.6g
53.6
g
g 99mmx30mmx14mm(onlyhousing)|108mmx30mmx14mm(includingtheconnector
['99', '30', '14']
mm
g 10.6oz.(300grams
300
g
g 3.9”x3.9”x1.7”(99mmx99mmx43mm
['99', '99', '43']
mm
g 1.1oz(32.0g
32.0
g
g 3.4”x1.2”x0.5”(85.9mmx30.0mmx12.6mm
['85.9', '30.0', '12.6']
mm
g 3.5oz.(98grams
98
g
g 3.2”x1.5”x2.2”(80mmx38mmx57mm
['80', '38', '57']
mm
g 8.8ounces
249.48000000000005
g
g 8.8ounces
249.48000000000005
g
g 3.7x0.8x0.5inches
['93.98', '20.32', '12.7']
mm
g 4ounces
113.4
g
g 4ounces
113.4
g
g 0.4x3.9x7.1inches
['10.16', '99.05999999999999', '180.33999999999997']
mm
g 10.6oz.(300grams
300
g
g 3.9”x3.9”x1.7”(99mmx99mmx43mm
['99', '99', '43']
mm
g 10.6oz.(300grams
300
g
g 3.9”x3.9”x1.7”(99mmx99mmx43mm
['99', '99', '43']
mm
g 3.2ounces
90.72000000000001
g
g 7.2ounces
204.12
g
g 2.2x2x2inches
['55.88', '50.8', '50.8']
mm
g 1.6oz(45grams
45
g
g 3.3”x1.9”x0.5”(85mmx47mmx13.28mm
['85', '47', '13.28']
mm
g 3.2ounces
90.72000000000001
g
g 4.8ounces
136.08
g
g 2.8x1.7x1inches
['71.119999999999

In [0]:
#--------------------Data Cleaning-----------------------------#
my_data=pd.read_excel("data00.xlsx")# read excel

#merge dimensions AND item dimensions AND size
my_data["dimension"]=my_data["Dimensions"].map(str)+my_data["Item Dimensions  L x W x H"].map(str)+my_data["Size"].map(str)
my_data["dimension"]=my_data["dimension"].str.replace("nan","")#replace NAN after merge
for i,value in enumerate(my_data["dimension"]):#对split dimensions
    if value!="":
        my_data.loc[i,'length']=float(eval(value)[0])
        my_data.loc[i, 'width'] = float(eval(value)[1])
        try:#because some dimension contains only 2 values, use try to fill the third as 0
            my_data.loc[i, 'height'] = float(eval(value)[2])
        except:
            my_data.loc[i, 'height'] = 0
    else:#if null in dimensions fill it 0
        my_data.loc[i,'length']=0
        my_data.loc[i, 'width'] = 0
        my_data.loc[i, 'height'] = 0
#del prior 3 columns
my_data.drop(["Dimensions","Item Dimensions  L x W x H","Size","dimension"],inplace=True,axis=1)

In [0]:
#sharpness generate 30%of length = 1
sharpness=[1]*int(len(my_data)*0.3)
sharpness.extend([0]*int(len(my_data)*0.7))#fill the rest with 0
np.random.shuffle(sharpness)#shuffle

liquid=[1]*int(len(my_data)*0.3)
liquid.extend([0]*int(len(my_data)*0.7))
np.random.shuffle(liquid)# the same

fragile=[1]*int(len(my_data)*0.3)
fragile.extend([0]*int(len(my_data)*0.7))
np.random.shuffle(fragile)#the same

my_data["sharpness"]=pd.Series(sharpness)
my_data["liquid"]=pd.Series(liquid)
my_data["fragile"]=pd.Series(fragile)#add 3 columns to mydata

my_data.fillna(0,inplace=True)# fill NA with 0 in mydata

print(my_data.head())
my_data.to_excel("data01.xlsx",header=True, index=False)#save

                                            name  Shipping Weight  \
0       Fire-TV-Stick-4K-with-Alexa-Voice-Remote             0.00   
1                                       Echo-Dot             0.00   
2          Fire-TV-Stick-with-Alexa-Voice-Remote             0.00   
3                  Amazon-Smart-Plug-works-Alexa             0.00   
4  Roku-Streaming-Stick-HDR-Streaming-Long-range           249.48   

   Item Weight Package Dimensions          Product Dimensions  Weight  length  \
0         0.00                  0                           0    53.6    99.0   
1         0.00                  0                           0   300.0    99.0   
2         0.00                  0                           0    32.0    85.9   
3         0.00                  0                           0    98.0    80.0   
4       249.48                  0  ['93.98', '20.32', '12.7']     0.0     0.0   

   width  height  sharpness  liquid  fragile  
0   30.0    14.0        0.0     0.0      0.0  
1   